In [ ]:
# install the necessary libraries
! pip install numpy
! pip install matplotlib
! pip install -U scikit-learn
! pip install pandas
! pip install -U imbalanced-learn
! pip install Bio

In [ ]:
# Download and install Miniconda
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
!bash miniconda.sh -bfp /usr/local

# Add conda to your PATH
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')

# Update conda
!conda update -q -y conda

In [ ]:
# Install the CD-HIT Algorithm

! conda install -c bioconda cd-hit -y

In [ ]:
import pandas as pd
import os
import requests

#Assign the positive and negative files
positive_file_url = "https://raw.githubusercontent.com/SanaeEsskhayry/data/master/AMR_protein_sequences.fasta"
negative_file_url = "https://raw.githubusercontent.com/SanaeEsskhayry/data/master/Sen_protein_sequences.fasta"

#Create a vairbale:filename dictionary
url_dict = {positive_file_url:'resistant_sequences.fasta', negative_file_url:'sensitive_sequences.fasta'}

#Loop through the variables in the dictionary and save the contents as filenames in the dictionary
for url, filename in url_dict.items():
  r = requests.get(url)
  with open (f'{filename}', 'wb') as f:
    f.write(r.content)



In [ ]:
import re
from Bio import SeqIO

# Function to find and return the ambiguous characters along with their positions in a protein sequence
def find_ambiguous_characters(sequence):
    ambiguous_pattern = re.compile(r"[^ACDEFGHIKLMNPQRSTVWY]")
    matches = [(match.group(), match.start()) for match in ambiguous_pattern.finditer(sequence)]
    return matches

# Function to check if a sequence has ambiguous characters
def has_ambiguous_characters(sequence):
    return bool(find_ambiguous_characters(sequence))

# Read the FASTA file and check for ambiguous characters
def check_ambiguous_characters(fasta_file):
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence_id = record.id
        sequence = str(record.seq)
        ambiguous_characters = find_ambiguous_characters(sequence)
        if ambiguous_characters:
            print(f"Ambiguous characters found in sequence {sequence_id}: {sequence}")
            for char, position in ambiguous_characters:
                print(f"Ambiguous character '{char}' found at position {position}")

# Read the FASTA file and filter out sequences with ambiguous characters
def filter_ambiguous_sequences(input_fasta, output_fasta):
    sequences_to_keep = []

    for record in SeqIO.parse(input_fasta, "fasta"):
        sequence = str(record.seq)
        if not has_ambiguous_characters(sequence):
            sequences_to_keep.append(record)

    # Write the filtered sequences to a new FASTA file
    with open(output_fasta, "w") as output_file:
        SeqIO.write(sequences_to_keep, output_file, "fasta")

# Input and output FASTA file names
input_fasta_file = "sensitive_sequences.fasta"
output_fasta_file = "filtered_sensitive_sequences.fasta"

# Check for ambiguous characters in the sequences
check_ambiguous_characters(input_fasta_file)

# Filter out sequences with ambiguous characters
filter_ambiguous_sequences(input_fasta_file, output_fasta_file)

In [ ]:
import re
from Bio import SeqIO

# Function to find and return the ambiguous characters along with their positions in a protein sequence
def find_ambiguous_characters(sequence):
    ambiguous_pattern = re.compile(r"[^ACDEFGHIKLMNPQRSTVWY]")
    matches = [(match.group(), match.start()) for match in ambiguous_pattern.finditer(sequence)]
    return matches

# Function to check if a sequence has ambiguous characters
def has_ambiguous_characters(sequence):
    return bool(find_ambiguous_characters(sequence))

# Read the FASTA file and check for ambiguous characters
def check_ambiguous_characters(fasta_file):
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence_id = record.id
        sequence = str(record.seq)
        ambiguous_characters = find_ambiguous_characters(sequence)
        if ambiguous_characters:
            print(f"Ambiguous characters found in sequence {sequence_id}: {sequence}")
            for char, position in ambiguous_characters:
                print(f"Ambiguous character '{char}' found at position {position}")

# Read the FASTA file and filter out sequences with ambiguous characters
def filter_ambiguous_sequences(input_fasta, output_fasta):
    sequences_to_keep = []

    for record in SeqIO.parse(input_fasta, "fasta"):
        sequence = str(record.seq)
        if not has_ambiguous_characters(sequence):
            sequences_to_keep.append(record)

    # Write the filtered sequences to a new FASTA file
    with open(output_fasta, "w") as output_file:
        SeqIO.write(sequences_to_keep, output_file, "fasta")

# Input and output FASTA file names
input_fasta_file = "resistant_sequences.fasta"
output_fasta_file = "filtered_resistant_sequences.fasta"

# Check for ambiguous characters in the sequences
check_ambiguous_characters(input_fasta_file)

# Filter out sequences with ambiguous characters
filter_ambiguous_sequences(input_fasta_file, output_fasta_file)

In [ ]:
#Process the files with cd-hit
os.system("cd-hit -i filtered_resistant_sequences.fasta -o resistant_cdhit.txt -c 0.99")
os.system("cd-hit -i filtered_sensitive_sequences.fasta -o sensitive_cdhit.txt -c 0.99")

In [ ]:
# -------------------------------------------------------------------------------
# Features generation for resistant sequences
# -------------------------------------------------------------------------------
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO

fasta_file = "resistant_cdhit.txt"
results = [ ]

for i, record in enumerate(SeqIO.parse(fasta_file, "fasta")):

    analysed_protein = ProteinAnalysis(record.seq)
    # Calculate properties
    charge_at_pH = analysed_protein.charge_at_pH(7)
    #1-
    molecular_weight = analysed_protein.molecular_weight()
    #2-
    hydrophobicity = analysed_protein.gravy()
    #3-
    isoelectric_point = analysed_protein.isoelectric_point()
    #4-
    aromaticity = analysed_protein.aromaticity()
    #5-
    instability_index = analysed_protein.instability_index()
    #6-
    secondary_structure_fraction = analysed_protein.secondary_structure_fraction()
    #7-
    molar_extinction_coefficient = analysed_protein.molar_extinction_coefficient()
    #8-
    flexibility = analysed_protein.flexibility()
    # 9-
    get_amino_acids_percent = analysed_protein.get_amino_acids_percent()

    # Store the results
    results.append({
          "Sequence": i+1,
          "charge at pH =7" : charge_at_pH,
          "Molecular Weight": molecular_weight,
          "Hydrophobicity": hydrophobicity,
          "Isoelectric Point": isoelectric_point,
          "Aromaticity": aromaticity,
          "Instability Index": instability_index,
          "secondary structure fraction" : secondary_structure_fraction,
          "Molar extinction coefficient" : molar_extinction_coefficient,
          "Flexibility" : sum(flexibility) / len(flexibility),
          "Amino acids percent" : get_amino_acids_percent
    })



# Display results for each sequence
for i, result in enumerate(results):
    print(f"Sequence {i + 1} Results:")
    for key, value in result.items():
      print(f"{key}: {value}")
    print()

In [ ]:
import pandas as pd


# Process the data to create separate columns for Amino Acids Percent, Molar Extinction Coefficient, and Secondary Structure Fraction
for entry in results:
    # Convert Amino Acids Percent dictionary into separate columns
    for aa, percent in entry['Amino acids percent'].items():
        entry[f'{aa}'] = percent

    # Separate Molar Extinction Coefficient into separate columns
    entry['MEC_reduced cysteines'], entry['MEC_disulfid bridges'] = entry['Molar extinction coefficient']

    # Separate Secondary Structure Fraction into separate columns
    entry['SSF_Helix'], entry['SSF_Turn'], entry['SSF_Sheet'] = entry['secondary structure fraction']

# Convert 'results' into a DataFrame
df = pd.DataFrame(results)

# Drop the original columns that have been separated
df.drop(['Amino acids percent', 'Molar extinction coefficient', 'secondary structure fraction'], axis=1, inplace=True)

# Print the DataFrame
df


In [ ]:
# -------------------------------------------------------------------------
# Features generation for sensitive sequences :
# -------------------------------------------------------------------------


from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO

fasta_file = "sensitive_cdhit.txt"
results = [ ]

for i, record in enumerate(SeqIO.parse(fasta_file, "fasta")):

    analysed_protein = ProteinAnalysis(record.seq)
    # Calculate properties
    charge_at_pH = analysed_protein.charge_at_pH(7)
    #1-
    molecular_weight = analysed_protein.molecular_weight()
    #2-
    hydrophobicity = analysed_protein.gravy()
    #3-
    isoelectric_point = analysed_protein.isoelectric_point()
    #4-
    aromaticity = analysed_protein.aromaticity()
    #5-
    instability_index = analysed_protein.instability_index()
    #6-
    secondary_structure_fraction = analysed_protein.secondary_structure_fraction()
    #7-
    molar_extinction_coefficient = analysed_protein.molar_extinction_coefficient()
    #8-
    flexibility = analysed_protein.flexibility()
    # 9-
    get_amino_acids_percent = analysed_protein.get_amino_acids_percent()

    # Store the results
    results.append({
          "Sequence": i+1,
          "charge at pH =7" : charge_at_pH,
          "Molecular Weight": molecular_weight,
          "Hydrophobicity": hydrophobicity,
          "Isoelectric Point": isoelectric_point,
          "Aromaticity": aromaticity,
          "Instability Index": instability_index,
          "secondary structure fraction" : secondary_structure_fraction,
          "Molar extinction coefficient" : molar_extinction_coefficient,
          "Flexibility" : sum(flexibility) / len(flexibility),
          "Amino acids percent" : get_amino_acids_percent
    })



# Display results for each sequence
for i, result in enumerate(results):
    print(f"Sequence {i + 1} Results:")
    for key, value in result.items():
      print(f"{key}: {value}")
    print()

In [ ]:
import pandas as pd

# Process the data to create separate columns for Amino Acids Percent, Molar Extinction Coefficient, and Secondary Structure Fraction
for entry in results:
    # Convert Amino Acids Percent dictionary into separate columns
    for aa, percent in entry['Amino acids percent'].items():
        entry[f'{aa}'] = percent

    # Separate Molar Extinction Coefficient into separate columns
    entry['MEC_reduced cysteines'], entry['MEC_disulfid bridges'] = entry['Molar extinction coefficient']

    # Separate Secondary Structure Fraction into separate columns
    entry['SSF_Helix'], entry['SSF_Turn'], entry['SSF_Sheet'] = entry['secondary structure fraction']

# Convert 'results' into a DataFrame
Sen_df = pd.DataFrame(results)

# Drop the original columns that have been separated
Sen_df.drop(['Amino acids percent', 'Molar extinction coefficient', 'secondary structure fraction'], axis=1, inplace=True)

# Print the DataFrame
Sen_df


In [ ]:
import csv
# Create class labels
Sen_class = pd.Series(['Sensitive' for i in range(len(Sen_df))])
AMR_class = pd.Series(['Resistant' for i in range(len(df))])
# Combine AMR Data and Sensitive Data
AMR_Sen_class = pd.concat([Sen_class, AMR_class], axis=0)
AMR_Sen_class.name = 'class'
AMR_Sen_feature = pd.concat([Sen_df, df], axis=0)

# Combine feature and class
df = pd.concat([AMR_Sen_feature, AMR_Sen_class], axis=1)

In [ ]:
df

In [ ]:
# Assigns the features to X and class label to Y
X = df.drop('class', axis=1)
y = df['class'].copy()

In [ ]:
y.value_counts()

In [ ]:
# Show pie plot
y.value_counts().plot.pie(autopct='%.2f')

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy=1) # Numerical value
# rus = RandomUnderSampler(sampling_strategy="not minority") # String
X_res, y_res = rus.fit_resample(X, y)

ax = y_res.value_counts().plot.pie(autopct='%.2f')
_ = ax.set_title("Under-sampling")


In [ ]:
# Feature selection (Variance threshold)
from sklearn.feature_selection import VarianceThreshold

fs = VarianceThreshold(threshold=0.1)
fs.fit_transform(X_res)
#X2.shape
X2 = X_res.loc[:, fs.get_support()]
X2

In [ ]:
X2.shape, y_res.shape

In [ ]:
y_res

In [ ]:
# Encoding the Y class label
y_res = y_res.map({"Resistant": 1, "Sensitive": 0})

In [ ]:
# Save the Features list into a csv file to be use it in the app interface:

X2.to_csv('features_list.csv', index = False)

In [ ]:
import pandas as pd
nan_rows = pd.isna(y_res)
print(nan_rows)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Initialize classifiers
classifiers = [
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier()),
    ('XGBoost', XGBClassifier()),
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('SVM', SVC())
]

# Print hyperparameters for each classifier
for name, clf in classifiers:
    print(f"Hyperparameters for {name}:")
    print(clf.get_params())
    print()




In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X2, y_res, test_size=0.2, random_state=42, stratify=y_res)

# Initialize classifiers with hyperparameters for tuning
classifiers = [
    ('K-Nearest Neighbors', KNeighborsClassifier(),
     {'n_neighbors': [3, 5, 7, 9, 10 , 20, 30, 40, 50], 'weights': ['uniform', 'distance']}),

    ('Decision Tree', DecisionTreeClassifier(),
     {'max_depth': [None, 10, 20, 30, 40, 50], 'min_samples_split': [2, 5, 10,20,30]}),

    ('Random Forest', RandomForestClassifier(),
     {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}),

    ('Gradient Boosting Machine', GradientBoostingClassifier(),
     {'n_estimators': [50, 100, 200, 300], 'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5 , 1]}),

    ('XGBoost', XGBClassifier(),
     {'n_estimators': [50, 100, 200, 300], 'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5 , 1]}),

    ('Logistic Regression', LogisticRegression(max_iter=1000),
     {'C': [0.1, 1, 10 , 20, 30 , 40, 50]}),

    ('Support Vector Machine', SVC(),
     {'C': [0.1, 1, 10, 20, 30, 40, 50, 60,70]})
]

# Loop through classifiers
for name, clf, param_grid in classifiers:
    print(f"Training and evaluating {name}...")

    # Hyperparameter tuning using GridSearchCV
    grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Best hyperparameters and their accuracy
    best_params = grid_search.best_params_
    best_accuracy = grid_search.best_score_
    print(f"Best Parameters: {best_params}")
    print(f"Best Accuracy: {best_accuracy:.2f}")

    # Evaluate on test data
    clf_best = grid_search.best_estimator_
    y_pred = clf_best.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    # Calculate ROC AUC for binary classification
    if name == 'Support Vector Machine' or name == 'Logistic Regression':
        roc_auc = roc_auc_score(y_test, clf_best.decision_function(X_test))
    else:
        roc_auc = roc_auc_score(y_test, clf_best.predict_proba(X_test)[:, 1])

    print(f"{name} Accuracy: {accuracy:.2f}")
    print(f"{name} Precision: {precision:.2f}")
    print(f"{name} Recall: {recall:.2f}")
    print(f"{name} F1-score: {f1:.2f}")
    print(f"{name} AUROC: {roc_auc:.2f}")
    print()


In [ ]:
# -------------------------------------------------------------------------
# Train the best model
# -------------------------------------------------------------------------

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score



X_train, X_test, y_train, y_test = train_test_split(X2, y_res, test_size=0.2, random_state=42)
model = GradientBoostingClassifier(n_estimators=50, learning_rate=0.1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
# ----------------------------------------------------------------
# Save Model as Pickle Object
# ----------------------------------------------------------------

import pickle
with open('amr_predict_model.pkl', 'wb') as f:
    pickle.dump(model, f)